In [1]:
# True: Google Colab Notebook
# False: My local PC
colab = False
if colab: 
    from google.colab import drive
    drive.mount('/content/drive')
    !ls /content/drive/MyDrive/output/otto/
    base_path = '/content/drive/MyDrive'
    notebook_path = base_path + '/otto/notebook'
    !pip3 install optuna
else:
    base_path = '../data'
    notebook_path = '.'

# Preprocessing

In [2]:
# ====================================================
# Library
# ====================================================
import gc
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools
from imblearn.under_sampling import RandomUnderSampler
import sys
sys.path.append(f"{notebook_path}/../src/")
import feature_engineering as fe

## Comparison between tetsuro v.s. Drake

In [3]:
# my data
my_train = pd.read_parquet(f'{base_path}/output/otto/train_50_0_ver2.parquet')
my_train = my_train[['session', 'aid', 'y_orders']]

In [4]:
print('tetsuro candidate')
print(my_train.shape)
print('target sum:', my_train['y_orders'].sum())
print('target mean:', my_train['y_orders'].mean())

tetsuro candidate
(169059644, 3)
target sum: 224413
target mean: 0.0013274190971323705


In [5]:
train = np.load(f'{base_path}/validation/order_validation.pkl', allow_pickle=True)
train = train.rename(columns={'candidates': 'aid'})

In [6]:
print('drake candidate')
print(train.shape)
print('target sum:', train['candidate_labels'].sum())
print('target mean:', train['candidate_labels'].mean())

drake candidate
(181074889, 4)
target sum: 223087
target mean: 0.0012320151139233889


In [22]:
merged_train = train.merge(my_train, how = 'outer', on = ['session','aid'])
merged_train['candidate_labels'] = merged_train['candidate_labels'].fillna(0) 
merged_train['y_orders'] = merged_train['y_orders'].fillna(False) 
merged_train['label'] = merged_train['candidate_labels'] + merged_train['y_orders']

In [34]:
total = merged_train.shape[0]
pos = merged_train.query('label>0').shape[0]
print('merged candidate')
print(total)
print('target sum:', pos)
print('target mean:', pos/total)

merged candidate
280559942
target sum: 228642
target mean: 0.0008149488425542945


In [37]:
print('After candidate merge...')
print('num of candidate become:', 280559942/169059644, "times")
print('num of positive:', 228642/223087, "times")

After candidate merge...
num of candidate become: 1.659532312749931 times
num of positive: 1.024900599317755 times
